<a href="https://colab.research.google.com/github/qdrptd/RNE/blob/main/YTCrawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 983 kB 8.4 MB/s 
     |████████████████████████████████| 358 kB 39.4 MB/s 
     |████████████████████████████████| 138 kB 64.1 MB/s 
     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 4.0 MB 41.5 MB/s 
     |████████████████████████████████| 58 kB 6.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://d

In [ ]:
from selenium import webdriver
import time
from openpyxl import Workbook
import pandas as pd
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [ ]:
# -*- coding: UTF-8 -*-
import time
from selenium import webdriver
import requests
!pip install xlsxwriter
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
 
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 10.5 MB/s 


In [ ]:
#해당 url로 이동
pd_metadata = {}
pd_comments = pd.DataFrame({})
for t in range(1, 4):
  type_n = open('type'+str(t)+".txt").readlines()
  print(type_n)
  search_data = {}
  for search in type_n[2:]:
    print(search)
    driver = webdriver.Chrome('chromedriver', options=options)
    url = "https://www.youtube.com/results?search_query=" + search #search 단어를 검색했을 때 나오는 페이지
    driver.get(url)
    driver.execute_script("window.scrollTo(0,1000)")
    time.sleep(3)

    # 페이지 끝까지 스크롤(복사)
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    print(last_height)
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(1.5)

        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

        if new_height > 10000: #동영상은 꽤 많이 나와서, 페이지 길이에 한계를 설정
          break
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    video_list = soup.select("a#video-title")

    id, titles = [], []
    for video in video_list: 
      id.append(video['href']) #동영상 ID(링크)
      titles.append(video['title']) #동영상 제목
    views, channels, subs = [], [], []
    channel_id_list = soup.select('div#channel-info > a')
    views_list = soup.select("div#metadata > div > span:nth-of-type(1)")
    time_list = soup.select("div#metadata > div > span:nth-of-type(2)")

    for channel_id in channel_id_list:
      try:
        channels.append(channel_id['href'])
        channel_url = "https://www.youtube.com" + channel_id['href']
        channel_html = requests.get(channel_url).text
        channel_soup = BeautifulSoup(channel_html, 'html.parser')
        subs.append(channel_soup.select('#subscriber-count').text)
      except:
        pass
    
    for v in views_list:
      views.append(v.text)
    # search_data[search] = {"아이디" : id, "제목" : titles, "조회수": views, "채널": channels, "구독자": subs}
    id_final = []
    comment_final = []
    likes_final = []
    video_comments_data = {}
    for vid_idx in range(len(id)):
      video_id = id[vid_idx]
      url = "https://www.youtube.com" + video_id
      try:
        driver.get(url)
      except:
        break
      driver.implicitly_wait(3)

      time.sleep(1.5)

      driver.execute_script("window.scrollTo(0,1000)")
      time.sleep(3)

      # 페이지 끝까지 스크롤
      last_height = driver.execute_script("return document.documentElement.scrollHeight")

      while True:
          driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
          time.sleep(1.5)

          new_height = driver.execute_script("return document.documentElement.scrollHeight")
          if new_height == last_height:
              break
          last_height = new_height
          if new_height > 100000:
            break

      time.sleep(1.5)

      # 팝업 닫기
      try:
          driver.find_element_by_css_selector("#dismiss-button > a").click()
      except:
          pass

      # 대댓글 모두 열기
      buttons = driver.find_elements_by_css_selector("#more-replies > a")

      time.sleep(1.5)

      for button in buttons:
          try:
              button.send_keys(Keys.ENTER)
          except:
            pass
          time.sleep(1.5)
          try:
            button.click()
          except:
            pass

      time.sleep(1.5)

      # 정보 추출하기
      html_source = driver.page_source
      soup = BeautifulSoup(html_source, 'html.parser')

      id_list = soup.select("div#header-author > h3 > #author-text > span")
      comment_list = soup.select("yt-formatted-string#content-text")
      likes_list = soup.select('#vote-count-middle')
      for i in range(len(comment_list)):
        temp_id = id_list[i].text
        temp_id = temp_id.replace('\n', '')
        temp_id = temp_id.replace('\t', '')
        temp_id = temp_id.replace('    ', '')
        id_final.append(temp_id)

        temp_comment = comment_list[i].text
        temp_comment = temp_comment.replace('\n', '')
        temp_comment = temp_comment.replace('\t', '')
        temp_comment = temp_comment.replace('    ', '')
        comment_final.append(temp_comment)

        likes_final.append(likes_list[i].text)
      pd_comments = pd.concat([pd_comments, pd.DataFrame({'타입': [t for temp in range(len(id_final))], '검색어': [search for temp in range(len(id_final))],'영상': [video_id for temp in range(len(id_final))], \
                                                          # '구독자': [subs[vid_idx] for temp in range(len(id_final))],\
                                                          '작성자': id_final, '내용': comment_final\
                                                          ,'좋아요': likes_final
                                                          })])
      print("종료")
    break
  break
  # pd_metadata['type' + str(i)] = search_data
print(pd_comments)
pd.DataFrame(pd_comments).to_excel("comments.xlsx")
# pd_metadata = pd.DataFrame(pd_metadata)
# pd_metadata.to_excel('metadata.xlsx')
driver.quit()

['metaverse copyright problem\n', 'metaverse sex offense\n', 'metaverse bullying\n', 'money laundering in metaverse\n', '메타버스 코인 투기\n']
metaverse bullying

4447


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:92: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:97: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead


종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
종료
       타입                   검색어                    영상                 작성자  \
0       1  metaverse bullying\n  /watch?v=sUHugSKBryw         Damon Katos   
1       1  metaverse bullying\n  /watch?v=sUHugSKBryw           DRENS2131   
2       1  metaverse bullying\n  /watch?v=sUHugSKBryw        Jake Simpson   
3       1  metaverse bullying\n  /watch?v=sUHugSKBryw            DhiyaNet   
4       1  metaverse bullying\n  /watch?v=sUHugSKBryw            LOVEIT38   
...    ..                   ...                   ...                 ...   
20366   1  metaverse bullying\n  /watch?v=pU_9XPYwzFw    Jovanka Stamnova   
20367   1  metaverse bullying\n  /watch?v=pU_9XPYwzFw        Tanner Jones   
20368   1  metaverse bullying\n  /watch?v=pU_9XPYwzFw        Antonius Kok   
20369   1  metaverse bullying\n  /watch?v=pU_9XPYwzFw       Aleesa Blight   
20370   1  metaverse bullying\n  /watch?v=pU_9XPYwzFw    

In [ ]:
pd.DataFrame(pd_comments).to_excel("comments.xlsx")

In [ ]:
channel_url = "https://www.youtube.com/cnaconnect"
driver = webdriver.Chrome('chromedriver', options=options)
driver.get(channel_url)
driver.execute_script("window.scrollTo(0,1000)")
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(1.5)

    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    if new_height > 100000:
        break
try:
    driver.find_element_by_css_selector("#dismiss-button > a").click()
except:
    pass

channel_html = requests.get(channel_url).text
channel_soup = BeautifulSoup(channel_html, 'html.parser')
print(channel_soup.select('yt-formatted-string#subscriber-count'))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead


[]
